**INSTALACION DE DEPENDENCIAS**

In [ ]:
%pip install ultralytics opencv-python-headless

IMPORTACIÓN DE DEPENDENCIAS

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

CARGAR Y UTILIZAR MODELO

In [ ]:
# Cargar el modelo YOLOv8
model = YOLO("./best.pt")

# Directorios
input_folder = "./imagenes"
crop_folder = "./localización"
no_detected_folder = "./no_identificadas"
os.makedirs(crop_folder, exist_ok=True)
os.makedirs(no_detected_folder, exist_ok=True)

CROP_SIZE = 800  # Área original a extraer alrededor del centro
FINAL_SIZE = 512  # Tamaño final después del resize

# Procesar imágenes
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    image = cv2.imread(img_path)
    if image is None:
        print(f"No se pudo cargar: {img_name}")
        continue

    height, width, _ = image.shape
    results = model(img_path)

    boxes = results[0].boxes.xyxy.cpu().numpy()

    if len(boxes) == 0:
        # Si no se detectaron objetos, guardar en carpeta especial
        cv2.imwrite(os.path.join(no_detected_folder, img_name), image)
        print(f"No se detectó nada en: {img_name} → guardada en 'no_identificadas'")
        continue

    for idx, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = map(int, box[:4])

        # Centro del bounding box
        x_center = int((x_min + x_max) / 2)
        y_center = int((y_min + y_max) / 2)

        # Definir recorte centrado de CROP_SIZE
        half_crop = CROP_SIZE // 2
        x1 = x_center - half_crop
        y1 = y_center - half_crop
        x2 = x_center + half_crop
        y2 = y_center + half_crop

        # Crear imagen negra (para padding)
        cropped = np.zeros((CROP_SIZE, CROP_SIZE, 3), dtype=np.uint8)

        # Coordenadas válidas dentro de la imagen original
        x1_img = max(x1, 0)
        y1_img = max(y1, 0)
        x2_img = min(x2, width)
        y2_img = min(y2, height)

        # Coordenadas donde se insertará en la imagen negra
        x1_dst = x1_img - x1
        y1_dst = y1_img - y1
        x2_dst = x1_dst + (x2_img - x1_img)
        y2_dst = y1_dst + (y2_img - y1_img)

        # Insertar región válida
        cropped[y1_dst:y2_dst, x1_dst:x2_dst] = image[y1_img:y2_img, x1_img:x2_img]

        # Redimensionar a 512x512
        cropped_resized = cv2.resize(cropped, (FINAL_SIZE, FINAL_SIZE))

        # Guardar recorte
        crop_name = f"{os.path.splitext(img_name)[0]}.jpg"
        cv2.imwrite(os.path.join(crop_folder, crop_name), cropped_resized)

    print(f"Procesada y guardada: {img_name}")

print("Detecciones, recortes centrados y resize completados.")
